In [2]:
using BSON: @save
using BSON: @load
using CSV
using DataFrames: DataFrame
using Flux
using Flux: logitbinarycrossentropy, binarycrossentropy
using Flux.Data: DataLoader
using Flux: chunk
using ImageFiltering
using MLDatasets: FashionMNIST
using ProgressMeter: Progress, next!
using Random
using Zygote
using MLDatasets
using Images
using ImageIO

using NBInclude
@nbinclude("functions.ipynb")

All function imported


In [3]:
x_dim = 400
k = 10
F = dct(diagm(ones(x_dim)),2);
A = sample_fourier(k,x_dim); A = A'
print(A[1,1])


0.05

In [41]:
loss(F,A) = maximum(sqrt.(sum((A'*F').*(A'*F'), dims = 1))) + norm(A'*A - I(k),2)^2
A = sample_fourier(k,x_dim); A = A'
A = randn(x_dim,k)

opt = ADAM(.1)
θ = Flux.params(A)
gs = gradient(()->loss(F,A), θ)
error =[]
old_A = A
print(old_A[1,1])
for i in 1:100
    Flux.Optimise.update!(opt, A, gs[A]);
    push!(error, norm(old_A - A))
end

# gs = gradient(()-x_dim>loss(F), θ)
# Flux.Optimise.update!(opt, A, gs[A]);

# opt = Descent(.1)
# for i in 1:100
#     gs = gradient(()->loss(F,A), A)
#     Flux.Optimise.update!(opt, A, gs[A]);
#     # A = A + randn(400,10)*.5^i
#     # θ = Flux.params(A)

# end

0.21951643939866214

In [44]:
loss_2(F,A) = maximum(sqrt.(sum((F*A).*(F*A), dims = 2))) + norm(A'*A - I(k),2)^2


loss_2 (generic function with 1 method)

In [45]:
loss(F,A)

321431.4963954395

In [46]:
loss_2(F,A)

321431.4963954395